In [52]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, LeaveOneGroupOut, cross_validate, cross_val_predict
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')



In [3]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [4]:
test3_master_ready = pd.read_csv('./mIHC/test3/test3_master_ready_1.tsv', sep='\t')
test3_master_ready

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
0,SMA06F_55,SMA06F,A,0.562500,5.654545,8.128440,1.101010,1.133333,3.544601,9.411043,...,0.017857,0.036585,0.034188,0.000000,2744.925466,3.366460,0.088000,161,1.041667,0.076923
1,SMA06F_72,SMA06F,A,0.000000,13.121212,3.412844,0.050505,0.158974,0.023474,6.490798,...,0.116071,0.000000,0.267094,0.069124,3760.303704,3.755556,0.000000,135,0.158333,0.215385
2,SMA06F_74,SMA06F,A,0.000000,8.496970,4.779817,0.474747,1.835897,0.000000,11.196319,...,0.116071,0.000000,0.136752,0.556452,3828.129412,2.964706,0.000000,85,1.191667,0.000000
3,SMA06F_75,SMA06F,A,0.153409,7.551515,5.302752,0.494949,0.907692,0.018779,7.987730,...,0.053571,0.000000,0.196581,0.000000,3895.500000,2.300000,0.056000,40,1.516667,0.023077
4,SMA06F_76,SMA06F,A,1.772727,7.072727,3.596330,0.101010,0.676923,0.051643,9.257669,...,0.125000,0.000000,0.239316,0.004608,3927.212121,2.666667,0.008000,66,2.241667,0.469231
5,SMA06F_78,SMA06F,A,7.596591,4.072727,4.834863,1.929293,0.584615,0.403756,10.073620,...,0.006696,0.060976,0.051282,0.210829,981.397790,12.060773,1.424000,181,8.525000,0.484615
6,SMA06F_108,SMA06F,A,0.340909,6.448485,5.532110,0.424242,1.061538,0.394366,12.233129,...,0.044643,0.024390,0.111111,0.009217,3614.991525,10.042373,0.024000,118,0.441667,0.076923
7,SMA06F_152,SMA06F,A,0.369318,11.672727,10.477064,0.606061,10.841026,0.145540,16.662577,...,0.000000,0.000000,0.119658,0.004608,3780.918182,14.400000,0.000000,110,4.083333,0.046154
8,SMA06F_156,SMA06F,A,0.062500,3.593939,5.853211,0.737374,0.574359,0.023474,6.049080,...,0.008929,0.024390,0.042735,0.110599,929.989130,13.478261,0.328000,92,0.408333,0.115385
9,SMA06F_168,SMA06F,A,0.909091,3.290909,3.091743,2.050505,0.507692,0.934272,4.674847,...,0.035714,0.012195,0.000000,0.110599,942.573913,16.643478,0.976000,115,1.208333,0.169231


In [7]:
predicted_by_random = pd.read_csv('./mIHC/Galaxy7-[Model_Validation_on_data_1_and_data_6].tabular', sep='\t')
balanced_accuracy_score(test3_master_ready['class'].values, predicted_by_random['Predicted'].values)

0.9432260372615686

In [8]:
predicted_by_patient = pd.read_csv('./mIHC/Galaxy8-[Model_Validation_on_data_1_and_data_6].tabular', sep='\t')
balanced_accuracy_score(test3_master_ready['class'].values, predicted_by_patient['Predicted'].values)

0.8216115504975116

## Results analysis for splitting patients

In [12]:
combined_predict = test3_master_ready.iloc[:, :3].merge(predicted_by_patient, left_index=True, right_index=True)
combined_predict

,obj_ID,patient_ID,class,Predicted
0,SMA06F_55,SMA06F,A,A
1,SMA06F_72,SMA06F,A,A
2,SMA06F_74,SMA06F,A,A
3,SMA06F_75,SMA06F,A,A
4,SMA06F_76,SMA06F,A,A
5,SMA06F_78,SMA06F,A,A
6,SMA06F_108,SMA06F,A,A
7,SMA06F_152,SMA06F,A,A
8,SMA06F_156,SMA06F,A,A
9,SMA06F_168,SMA06F,A,A


In [27]:
test3_master_ready.groupby(['patient_ID'])['obj_ID'].count()

patient_ID
SMA06F    4175
SMA19F    3716
SMA21F    1950
SMA37F    4351
SMA40F    5637
SMA54F    3186
SMA60F    3990
Name: obj_ID, dtype: int64

In [60]:
combined_predict_SMA06F = combined_predict[combined_predict['patient_ID'] == 'SMA06F']
score_SMA06F  = balanced_accuracy_score(combined_predict_SMA06F['class'].values, combined_predict_SMA06F['Predicted'].values)

combined_predict_SMA21F = combined_predict[combined_predict['patient_ID'] == 'SMA21F']
score_SMA21F  = balanced_accuracy_score(combined_predict_SMA21F['class'].values, combined_predict_SMA21F['Predicted'].values)

combined_predict_SMA19F = combined_predict[combined_predict['patient_ID'] == 'SMA19F']
score_SMA19F  = balanced_accuracy_score(combined_predict_SMA19F['class'].values, combined_predict_SMA19F['Predicted'].values)

combined_predict_SMA37F = combined_predict[combined_predict['patient_ID'] == 'SMA37F']
score_SMA37F  = balanced_accuracy_score(combined_predict_SMA37F['class'].values, combined_predict_SMA37F['Predicted'].values)

combined_predict_SMA40F = combined_predict[combined_predict['patient_ID'] == 'SMA40F']
score_SMA40F  = balanced_accuracy_score(combined_predict_SMA40F['class'].values, combined_predict_SMA40F['Predicted'].values)

combined_predict_SMA54F = combined_predict[combined_predict['patient_ID'] == 'SMA54F']
score_SMA54F  = balanced_accuracy_score(combined_predict_SMA54F['class'].values, combined_predict_SMA54F['Predicted'].values)

combined_predict_SMA60F = combined_predict[combined_predict['patient_ID'] == 'SMA60F']
score_SMA60F  = balanced_accuracy_score(combined_predict_SMA60F['class'].values, combined_predict_SMA60F['Predicted'].values)

print("Accuracy per patient:")
score_SMA06F, score_SMA21F, score_SMA19F, score_SMA37F, score_SMA40F, score_SMA54F, score_SMA60F

Accuracy per patient:


(0.7764681201941203,
 0.7808504684954327,
 0.8803702559789082,
 0.8453847525897096,
 0.8332668662558456,
 0.9276200750426707,
 0.8109736760623139)

In [57]:
combined_predict_SMA06F = combined_predict[combined_predict['patient_ID'] == 'SMA06F']
score_SMA06F  = confusion_matrix(combined_predict_SMA06F['class'].values, combined_predict_SMA06F['Predicted'].values)

combined_predict_SMA21F = combined_predict[combined_predict['patient_ID'] == 'SMA21F']
score_SMA21F  = confusion_matrix(combined_predict_SMA21F['class'].values, combined_predict_SMA21F['Predicted'].values)

combined_predict_SMA19F = combined_predict[combined_predict['patient_ID'] == 'SMA19F']
score_SMA19F  = confusion_matrix(combined_predict_SMA19F['class'].values, combined_predict_SMA19F['Predicted'].values)

combined_predict_SMA37F = combined_predict[combined_predict['patient_ID'] == 'SMA37F']
score_SMA37F  = confusion_matrix(combined_predict_SMA37F['class'].values, combined_predict_SMA37F['Predicted'].values)

combined_predict_SMA40F = combined_predict[combined_predict['patient_ID'] == 'SMA40F']
score_SMA40F  = confusion_matrix(combined_predict_SMA40F['class'].values, combined_predict_SMA40F['Predicted'].values)

combined_predict_SMA54F = combined_predict[combined_predict['patient_ID'] == 'SMA54F']
score_SMA54F  = confusion_matrix(combined_predict_SMA54F['class'].values, combined_predict_SMA54F['Predicted'].values)

combined_predict_SMA60F = combined_predict[combined_predict['patient_ID'] == 'SMA60F']
score_SMA60F  = confusion_matrix(combined_predict_SMA60F['class'].values, combined_predict_SMA60F['Predicted'].values)

print("Confusion matrix by patients: ")
score_SMA06F, score_SMA21F, score_SMA19F, score_SMA37F, score_SMA40F, score_SMA54F, score_SMA60F

Confusion matrix by patients: 


(array([[551,   0,   0,   0,   0,   0,   0,   0],
        [ 32, 661,   0,   0,   0,   0,   0,   0],
        [  8,   1,  23,   0,   0,   0,   0,   0],
        [ 43,  63,   3, 368,  19,   0,   0,   0],
        [ 74,  75,   0,   0, 304,   0,   0,   0],
        [  4,  16,   0,  16,  21, 439,   0, 154],
        [  7,  13,   2,  11,  20, 175, 320, 102],
        [  1,   7,   1,   6,  12,   0,   0, 623]]),
 array([[ 88,   6,   0,   0,   2,   4,   2,   0],
        [  0, 126,   1,   0,   0,   0,   0,   0],
        [  0,   0,   4,   0,   0,   0,   0,   0],
        [  1,   2,   0,  21,   6,   0,   0,   0],
        [ 20,  37,   0,   0, 282,   5,  17,   0],
        [  0,   1,   0,   0,   2, 592,  55,   0],
        [  0,   0,   0,   0,   0,   0, 650,   0],
        [  2,   0,   0,   0,   1,  19,   4,   0]]),
 array([[140,  23,   0,   0,   0,   1,   0,   1],
        [  0, 428,   0,   1,   0,   0,   0,   3],
        [  0,   2,  65,   0,   0,   0,   0,   0],
        [  0,  13,   3, 114,  54,   2,   0,  1

In [55]:
combined_predict_A = combined_predict[combined_predict['class'] == 'A']
score_A  = accuracy_score(combined_predict_A['class'].values, combined_predict_A['Predicted'].values)

combined_predict_B = combined_predict[combined_predict['class'] == 'B']
score_B  = balanced_accuracy_score(combined_predict_B['class'].values, combined_predict_B['Predicted'].values)

combined_predict_C = combined_predict[combined_predict['class'] == 'C']
score_C  = balanced_accuracy_score(combined_predict_C['class'].values, combined_predict_C['Predicted'].values)

combined_predict_D = combined_predict[combined_predict['class'] == 'D']
score_D  = balanced_accuracy_score(combined_predict_D['class'].values, combined_predict_D['Predicted'].values)

combined_predict_E = combined_predict[combined_predict['class'] == 'E']
score_E  = balanced_accuracy_score(combined_predict_E['class'].values, combined_predict_E['Predicted'].values)

combined_predict_F = combined_predict[combined_predict['class'] == 'F']
score_F  = balanced_accuracy_score(combined_predict_F['class'].values, combined_predict_F['Predicted'].values)

combined_predict_G = combined_predict[combined_predict['class'] == 'G']
score_G  = balanced_accuracy_score(combined_predict_G['class'].values, combined_predict_G['Predicted'].values)

combined_predict_H = combined_predict[combined_predict['class'] == 'H']
score_H  = balanced_accuracy_score(combined_predict_H['class'].values, combined_predict_H['Predicted'].values)

print('Accuracy for class A-H:')
score_A, score_B, score_C, score_D, score_E, score_F, score_G, score_H

Accuracy for class A-H:


(0.8813223140495868,
 0.8327776352911003,
 0.7087378640776699,
 0.7706685837526959,
 0.7720008092251669,
 0.8705494505494505,
 0.9105494505494506,
 0.826286296484972)

In [59]:
combined_predict_A = combined_predict[combined_predict['class'] == 'A']
score_A  = confusion_matrix(combined_predict_A['class'].values, combined_predict_A['Predicted'].values)

combined_predict_B = combined_predict[combined_predict['class'] == 'B']
score_B  = confusion_matrix(combined_predict_B['class'].values, combined_predict_B['Predicted'].values)

combined_predict_C = combined_predict[combined_predict['class'] == 'C']
score_C  = confusion_matrix(combined_predict_C['class'].values, combined_predict_C['Predicted'].values)

combined_predict_D = combined_predict[combined_predict['class'] == 'D']
score_D  = confusion_matrix(combined_predict_D['class'].values, combined_predict_D['Predicted'].values)

combined_predict_E = combined_predict[combined_predict['class'] == 'E']
score_E  = confusion_matrix(combined_predict_E['class'].values, combined_predict_E['Predicted'].values)

combined_predict_F = combined_predict[combined_predict['class'] == 'F']
score_F  = confusion_matrix(combined_predict_F['class'].values, combined_predict_F['Predicted'].values)

combined_predict_G = combined_predict[combined_predict['class'] == 'G']
score_G  = confusion_matrix(combined_predict_G['class'].values, combined_predict_G['Predicted'].values)

combined_predict_H = combined_predict[combined_predict['class'] == 'H']
score_H  = confusion_matrix(combined_predict_H['class'].values, combined_predict_H['Predicted'].values)

print('Confusion matrix for class A-H:')
score_A, score_B, score_C, score_D, score_E, score_F, score_G, score_H

Confusion matrix for class A-H:


(array([[2666,   62,   19,   39,  172,   24,   17,   26],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]]),
 array([[   0,    0,    0,    0,    0,    0,    0,    0],
        [  67, 3247,   30,  103,  323,   78,   16,   35],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]]),
 array([[  0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,

## Random split result analysis

In [62]:
combined_predict = test3_master_ready.iloc[:, :3].merge(predicted_by_random, left_index=True, right_index=True)
combined_predict

,obj_ID,patient_ID,class,Predicted
0,SMA06F_55,SMA06F,A,A
1,SMA06F_72,SMA06F,A,A
2,SMA06F_74,SMA06F,A,A
3,SMA06F_75,SMA06F,A,A
4,SMA06F_76,SMA06F,A,A
5,SMA06F_78,SMA06F,A,A
6,SMA06F_108,SMA06F,A,A
7,SMA06F_152,SMA06F,A,A
8,SMA06F_156,SMA06F,A,A
9,SMA06F_168,SMA06F,A,A


In [63]:
test3_master_ready.groupby(['patient_ID'])['obj_ID'].count()

patient_ID
SMA06F    4175
SMA19F    3716
SMA21F    1950
SMA37F    4351
SMA40F    5637
SMA54F    3186
SMA60F    3990
Name: obj_ID, dtype: int64

In [64]:
combined_predict_SMA06F = combined_predict[combined_predict['patient_ID'] == 'SMA06F']
score_SMA06F  = balanced_accuracy_score(combined_predict_SMA06F['class'].values, combined_predict_SMA06F['Predicted'].values)

combined_predict_SMA21F = combined_predict[combined_predict['patient_ID'] == 'SMA21F']
score_SMA21F  = balanced_accuracy_score(combined_predict_SMA21F['class'].values, combined_predict_SMA21F['Predicted'].values)

combined_predict_SMA19F = combined_predict[combined_predict['patient_ID'] == 'SMA19F']
score_SMA19F  = balanced_accuracy_score(combined_predict_SMA19F['class'].values, combined_predict_SMA19F['Predicted'].values)

combined_predict_SMA37F = combined_predict[combined_predict['patient_ID'] == 'SMA37F']
score_SMA37F  = balanced_accuracy_score(combined_predict_SMA37F['class'].values, combined_predict_SMA37F['Predicted'].values)

combined_predict_SMA40F = combined_predict[combined_predict['patient_ID'] == 'SMA40F']
score_SMA40F  = balanced_accuracy_score(combined_predict_SMA40F['class'].values, combined_predict_SMA40F['Predicted'].values)

combined_predict_SMA54F = combined_predict[combined_predict['patient_ID'] == 'SMA54F']
score_SMA54F  = balanced_accuracy_score(combined_predict_SMA54F['class'].values, combined_predict_SMA54F['Predicted'].values)

combined_predict_SMA60F = combined_predict[combined_predict['patient_ID'] == 'SMA60F']
score_SMA60F  = balanced_accuracy_score(combined_predict_SMA60F['class'].values, combined_predict_SMA60F['Predicted'].values)

print("Accuracy per patient:")
score_SMA06F, score_SMA21F, score_SMA19F, score_SMA37F, score_SMA40F, score_SMA54F, score_SMA60F

Accuracy per patient:


(0.8858525463593825,
 0.8355387346285653,
 0.9488439139431548,
 0.9481245161973033,
 0.9408601508451755,
 0.9320093641591272,
 0.9543252742189141)

In [65]:
combined_predict_SMA06F = combined_predict[combined_predict['patient_ID'] == 'SMA06F']
score_SMA06F  = confusion_matrix(combined_predict_SMA06F['class'].values, combined_predict_SMA06F['Predicted'].values)

combined_predict_SMA21F = combined_predict[combined_predict['patient_ID'] == 'SMA21F']
score_SMA21F  = confusion_matrix(combined_predict_SMA21F['class'].values, combined_predict_SMA21F['Predicted'].values)

combined_predict_SMA19F = combined_predict[combined_predict['patient_ID'] == 'SMA19F']
score_SMA19F  = confusion_matrix(combined_predict_SMA19F['class'].values, combined_predict_SMA19F['Predicted'].values)

combined_predict_SMA37F = combined_predict[combined_predict['patient_ID'] == 'SMA37F']
score_SMA37F  = confusion_matrix(combined_predict_SMA37F['class'].values, combined_predict_SMA37F['Predicted'].values)

combined_predict_SMA40F = combined_predict[combined_predict['patient_ID'] == 'SMA40F']
score_SMA40F  = confusion_matrix(combined_predict_SMA40F['class'].values, combined_predict_SMA40F['Predicted'].values)

combined_predict_SMA54F = combined_predict[combined_predict['patient_ID'] == 'SMA54F']
score_SMA54F  = confusion_matrix(combined_predict_SMA54F['class'].values, combined_predict_SMA54F['Predicted'].values)

combined_predict_SMA60F = combined_predict[combined_predict['patient_ID'] == 'SMA60F']
score_SMA60F  = confusion_matrix(combined_predict_SMA60F['class'].values, combined_predict_SMA60F['Predicted'].values)

print("Confusion matrix by patients: ")
score_SMA06F, score_SMA21F, score_SMA19F, score_SMA37F, score_SMA40F, score_SMA54F, score_SMA60F

Confusion matrix by patients: 


(array([[551,   0,   0,   0,   0,   0,   0,   0],
        [ 31, 662,   0,   0,   0,   0,   0,   0],
        [  4,   1,  26,   1,   0,   0,   0,   0],
        [ 12,  13,   3, 461,   7,   0,   0,   0],
        [ 46,  44,   2,   4, 357,   0,   0,   0],
        [  4,   7,   0,  14,  21, 541,   0,  63],
        [  3,   8,   0,  10,   9,  87, 516,  17],
        [  0,   0,   0,   5,  11,   0,   0, 634]]),
 array([[ 93,   6,   0,   0,   1,   0,   2,   0],
        [  0, 125,   1,   0,   1,   0,   0,   0],
        [  0,   0,   4,   0,   0,   0,   0,   0],
        [  0,   0,   0,  21,   9,   0,   0,   0],
        [  3,   7,   0,   0, 344,   0,   7,   0],
        [  0,   0,   0,   0,   4, 613,  33,   0],
        [  0,   0,   0,   0,   0,   0, 650,   0],
        [  2,   0,   0,   0,   2,  14,   3,   5]]),
 array([[142,  23,   0,   0,   0,   0,   0,   0],
        [  0, 431,   0,   0,   1,   0,   0,   0],
        [  0,   2,  64,   1,   0,   0,   0,   0],
        [  0,   2,   1, 181,  18,   0,   0,   

In [66]:
combined_predict_A = combined_predict[combined_predict['class'] == 'A']
score_A  = accuracy_score(combined_predict_A['class'].values, combined_predict_A['Predicted'].values)

combined_predict_B = combined_predict[combined_predict['class'] == 'B']
score_B  = balanced_accuracy_score(combined_predict_B['class'].values, combined_predict_B['Predicted'].values)

combined_predict_C = combined_predict[combined_predict['class'] == 'C']
score_C  = balanced_accuracy_score(combined_predict_C['class'].values, combined_predict_C['Predicted'].values)

combined_predict_D = combined_predict[combined_predict['class'] == 'D']
score_D  = balanced_accuracy_score(combined_predict_D['class'].values, combined_predict_D['Predicted'].values)

combined_predict_E = combined_predict[combined_predict['class'] == 'E']
score_E  = balanced_accuracy_score(combined_predict_E['class'].values, combined_predict_E['Predicted'].values)

combined_predict_F = combined_predict[combined_predict['class'] == 'F']
score_F  = balanced_accuracy_score(combined_predict_F['class'].values, combined_predict_F['Predicted'].values)

combined_predict_G = combined_predict[combined_predict['class'] == 'G']
score_G  = balanced_accuracy_score(combined_predict_G['class'].values, combined_predict_G['Predicted'].values)

combined_predict_H = combined_predict[combined_predict['class'] == 'H']
score_H  = balanced_accuracy_score(combined_predict_H['class'].values, combined_predict_H['Predicted'].values)

print('Accuracy for class A-H:')
score_A, score_B, score_C, score_D, score_E, score_F, score_G, score_H

Accuracy for class A-H:


(0.9593388429752067,
 0.9422928956142601,
 0.9375866851595007,
 0.9367361610352265,
 0.9261582035201295,
 0.9120879120879121,
 0.963956043956044,
 0.9676515537442689)

In [67]:
combined_predict_A = combined_predict[combined_predict['class'] == 'A']
score_A  = confusion_matrix(combined_predict_A['class'].values, combined_predict_A['Predicted'].values)

combined_predict_B = combined_predict[combined_predict['class'] == 'B']
score_B  = confusion_matrix(combined_predict_B['class'].values, combined_predict_B['Predicted'].values)

combined_predict_C = combined_predict[combined_predict['class'] == 'C']
score_C  = confusion_matrix(combined_predict_C['class'].values, combined_predict_C['Predicted'].values)

combined_predict_D = combined_predict[combined_predict['class'] == 'D']
score_D  = confusion_matrix(combined_predict_D['class'].values, combined_predict_D['Predicted'].values)

combined_predict_E = combined_predict[combined_predict['class'] == 'E']
score_E  = confusion_matrix(combined_predict_E['class'].values, combined_predict_E['Predicted'].values)

combined_predict_F = combined_predict[combined_predict['class'] == 'F']
score_F  = confusion_matrix(combined_predict_F['class'].values, combined_predict_F['Predicted'].values)

combined_predict_G = combined_predict[combined_predict['class'] == 'G']
score_G  = confusion_matrix(combined_predict_G['class'].values, combined_predict_G['Predicted'].values)

combined_predict_H = combined_predict[combined_predict['class'] == 'H']
score_H  = confusion_matrix(combined_predict_H['class'].values, combined_predict_H['Predicted'].values)

print('Confusion matrix for class A-H:')
score_A, score_B, score_C, score_D, score_E, score_F, score_G, score_H

Confusion matrix for class A-H:


(array([[2902,   60,    8,    7,   39,    1,    5,    3],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]]),
 array([[   0,    0,    0,    0,    0,    0,    0,    0],
        [  54, 3674,   20,   31,  103,    7,    3,    7],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]]),
 array([[  0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,